# Example 1 - Plot power spectra

`PyCurious` offers convenience functions to extract the radial power spectrum. This has been derived analytically by Bouligand *et al.* 2009.

In this notebook we plot the radial power spectrum using the analytical expression, and compare it to the spectrum computed from a synthetic magnetic anomaly. (This can be generated from `Bouligand_forward.py` in the `tests` directory.)

### Contents

- [Analytical solution](#Analytical-solution)
- [Radial power spectrum](#Power-spectrum-from-FFT)
- [Azimuthal power spectrum](#Azimuthal-power-spectrum)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pycurious

## Analytical solution

The analytic solution to the power spectrum is given in Equation 4 of [Bouligand *et al.*, 2009](http://doi.wiley.com/10.1029/2009JB006494).

where the shape of the curve depends on 4 variables:

- $\beta$ - a fractal parameter
- $z_t$ - top of magnetic sources
- $\Delta z$ - thickness of the magnetic layer
- $C$ - a field constant

In [ ]:
# wavenumber range
k = np.linspace(1e-3, 3, 10000)

# Bouligand 2009 parameters
beta = 3.0
zt = 1.0
dz = 20.0

In [ ]:
# plot Bouligand et al. 2009 curves

fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(14,8),)

# vary zt
for zti in np.arange(0.0, 2.5, 0.5):
    S = pycurious.bouligand2009(beta, zti, dz, k)
    S -= S.max()
    ax1.semilogx(k, S, label=r'$z_t$ = {} km'.format(zti))

# vary dz
for dzi in [10., 20., 50., 100., 200.]:
    S = pycurious.bouligand2009(beta, zt, dzi, k)
    S -= S.min()
    ax2.semilogx(k, S, label=r'$\Delta z$ = {} km'.format(dzi))

# vary beta
for betai in np.arange(0, 5, 1):
    S = pycurious.bouligand2009(betai, zt, dz, k)
    S -= S[-1]
    ax3.semilogx(k, S, label=r'$\beta$ = {}'.format(betai))

ax1.set_ylim(-20, 0)
ax2.set_ylim(0, 20)

ax1.legend()
ax2.legend()
ax3.legend()

## Radial power spectrum

The radial power spectrum is computed from a square window of the magnetic anomaly. Methods to select window sizes and compute the Fast Fourier Transform (FFT) belong to the `CurieGrid` object.

`CurieGrid` achieves the following purposes:

- Upward continuation
- Reduction to the pole
- Compute the radial power spectrum using FFT

The shape of the radial power spectrum is heavily dependent on window size. Resolution of long wavelength features require large windows.

> Suggestion: use a window size **> 4 times** the maximum Curie depth.

In [ ]:
# load magnetic anomaly
mag_data = np.loadtxt("../data/test_mag_data.txt")

nx, ny = 305, 305

x = mag_data[:,0]
y = mag_data[:,1]
d = mag_data[:,2].reshape(ny,nx)

xmin, xmax = x.min(), x.max()
ymin, ymax = y.min(), y.max()

In [ ]:
# initialise CurieGrid object
grid = pycurious.CurieGrid(d, xmin, xmax, ymin, ymax)

# pick the centroid
xpt = 0.5*(xmin + xmax)
ypt = 0.5*(ymin + ymax)

window_size = 304e3
subgrid = grid.subgrid(xpt, ypt, window_size)

In [ ]:
# compute radial power spectrum
S, k, sigma2 = grid.radial_spectrum(subgrid)


# plot radial power spectrum
fig = plt.figure()
ax1 = fig.add_subplot(111, xlabel="wavenumber (rad/km)", ylabel="radial power spectrum")
ax1.plot(k, S, '-o')
plt.show()

The default taper is the hanning filter (see [`numpy.hanning`](#hanning) for more details), but other functions can be passed to taper, or simply set it to `None`.

In [ ]:

S, k, sigma2 = grid.radial_spectrum(subgrid, taper=None)


fig = plt.figure()
ax1 = fig.add_subplot(111, xlabel="wavenumber (rad/km)", ylabel="radial power spectrum")
ax1.plot(k, S, '-o')
plt.show()

## Azimuthal power spectrum

The azimuthal spectrum computes the FFT on a square window that is polarised at a range of radii. This is useful to explore linear trends in the magnetic anomaly that may align with a particular foliation or strike direction.

```python
azimuthal_spectrum(subgrid, theta=5.0)
```

`theta` controls the thickness of each azimuth (in degrees).

In [ ]:
S, k, theta = grid.azimuthal_spectrum(subgrid, theta=20.0)

fig = plt.figure()
ax1 = fig.add_subplot(111, ylabel=r'azimuth $\theta$', xlabel='wavenumber (rad/km)')
im1 = ax1.pcolor(k, theta, S)
fig.colorbar(im1, label='radial power spectrum')

Plotting each azimuth on the power spectrum-wavenumber axis highlights the different in slope for different polarisations of the magnetic anomaly.

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111, xlabel="wavenumber (rad/km)", ylabel="radial power spectrum")

for i, angle in enumerate(theta):
    ax1.plot(k, S[i], label=r'$\theta = {}^\circ$'.format(angle))

ax1.legend(bbox_to_anchor=(1,1))
plt.show()